In [ ]:
import pandas as pd
import numpy as np

# 1. Chargement des données 
try:
    df = pd.read_csv('train.csv')
except FileNotFoundError:
    print("Erreur : Le fichier 'train.csv' est introuvable.")
    exit()

# 2. Feature Engineering 

# Segmentation Démographique (Segment_Age) 
# Divise les passagers en Jeunes, Actifs et Seniors
df['Segment_Age'] = pd.cut(df['Age'], bins=[0, 25, 60, 100], 
                           labels=['Jeune (<25)', 'Actif (25-60)', 'Senior (>60)'])

# Classification des Vols (Type_Vol)
# Catégorise selon la distance du vol
df['Type_Vol'] = pd.cut(df['Flight Distance'], bins=[0, 800, 2500, 30000], 
                        labels=['Court-Courrier', 'Moyen-Courrier', 'Long-Courrier'])

# Prédiction du Motif de Voyage (Motif_Voyage_Predit) 
# Règle heuristique basée sur l'âge (Précision estimée : 70-75%)
def predict_purpose(row):
    return "Business (Predit)" if 30 <= row['Age'] <= 60 else "Loisir (Predit)"

df['Motif_Voyage_Predit'] = df.apply(predict_purpose, axis=1)

# Score de Risque Churn (Niveau_Risque_Churn) 
# Basé sur la classe, le type de vol et la fidélité
def calculate_risk(row):
    score = 0
    if row['Class'] == 'Eco': score += 2
    if row['Type_Vol'] == 'Long-Courrier': score += 1
    if row['Customer Type'] == 'disloyal Customer': score += 2
    
    if score >= 4: return "Risque Eleve"
    elif score >= 2: return "Risque Moyen"
    else: return "Risque Faible"

df['Niveau_Risque_Churn'] = df.apply(calculate_risk, axis=1)

# Attribution du Type d'Avion selon la flotte RAM 
def assign_aircraft(row):
    if row['Type_Vol'] == 'Long-Courrier': 
        return 'Boeing 787 Dreamliner'
    elif row['Type_Vol'] == 'Moyen-Courrier': 
        return 'Boeing 737-800'
    else: 
        return 'ATR 72/Embraer'

df['Type_Avion'] = df.apply(assign_aircraft, axis=1)

# Évaluation du Stress Aéroportuaire (Etat_Aeroport)
# Utilise la localisation de la porte comme indicateur de stress
def airport_stress(row):
    return "Aeroport Sature (Stress Eleve)" if row['Gate location'] <= 2 else "Aeroport Fluide"

df['Etat_Aeroport'] = df.apply(airport_stress, axis=1)

# 3. Traduction et Nettoyage 
df['Genre'] = df['Gender'].map({'Male': 'Homme', 'Female': 'Femme'})
df['Classe'] = df['Class'].map({'Business': 'Affaires', 'Eco': 'Eco', 'Eco Plus': 'Eco Premium'})
df['Satisfaction_Client'] = df['satisfaction'].map({
    'satisfied': 'Satisfait', 
    'neutral or dissatisfied': 'Insatisfait/Neutre'
})

# 4. Export du Dataset Final 
columns_to_keep = [
    'id', 'Genre', 'Age', 'Segment_Age', 'Classe', 'Type_Vol', 
    'Type_Avion', 'Etat_Aeroport', 'Motif_Voyage_Predit', 
    'Niveau_Risque_Churn', 'Satisfaction_Client', 'Seat comfort', 
    'Food and drink', 'Inflight wifi service', 'Departure Delay in Minutes', 
    'Gate location'
]

final_df = df[columns_to_keep]
final_df.to_csv('RAM_Simulation_Data_V2.csv', index=False)

print("Succès : Le dataset enrichi 'RAM_Simulation_Data_V2.csv' a été généré.")